# Notebook to Investigate LSTM Performance Variance by Huc Type

# Step 0 - Set up Notebook

In [1]:
# import needed libaries
import os
import boto3
import mlflow
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# initialize erathengine credentials
import ee
ee.Authenticate()

True

# Step1 - Download MLFlow Metrics

In [3]:
# function to retrieve metrics from ML server 
def load_ml_metrics(tracking_uri, run_id, save_local=False):
    mlflow.set_tracking_uri(tracking_uri)
    client = mlflow.MlflowClient()
    # Get all metric keys from the run
    run_data = client.get_run(run_id).data
    metric_keys = run_data.metrics.keys()
    # Retrieve full metric history for each key
    all_metrics = []
    for metric in metric_keys:
        history = client.get_metric_history(run_id, metric)
        for record in history:
            all_metrics.append({
                "Metric": metric,
                "Step": record.step,
                "Value": record.value
            })
    
    # Convert to DataFrame
    metrics_df = pd.DataFrame(all_metrics)
    
    # Save to CSV if needed
    if save_local:
        f_out = f"run_id_data/metrics_from_{run_id}.csv"
        metrics_df.to_csv(f_out, index=False)

    return metrics_df

In [4]:
# function to extract only a specific metric
def extract_metric(df, metric_name):
    """Extracts rows where the Metric column ends with 'metric_name' and returns only Metric and Value columns."""
    return df[df['Metric'].str.endswith(metric_name)][['Metric', 'Value']].sort_values(by='Metric')

In [5]:
# set mlFlow tracking server 
tracking_uri = "arn:aws:sagemaker:us-west-2:677276086662:mlflow-tracking-server/dawgsML"

# define our run_ids by recognizable names 
all_single = "69d929bbdfdd43b4a2f45b823d945eb7" # ~300 different huc12 units, 10 epochs each, dapper-pig-680 

In [6]:
df_all_single = load_ml_metrics(tracking_uri, all_single, save_local = True)

MlflowException: API request to https://us-west-2.experiments.sagemaker.aws/api/2.0/mlflow/runs/get failed with exception HTTPSConnectionPool(host='us-west-2.experiments.sagemaker.aws', port=443): Max retries exceeded with url: /api/2.0/mlflow/runs/get?run_uuid=69d929bbdfdd43b4a2f45b823d945eb7&run_id=69d929bbdfdd43b4a2f45b823d945eb7 (Caused by ResponseError('too many 503 error responses'))

In [ ]:
df_all_single.head()